### Instalação dos pacotes necessários

Pandas: Biblioteca para manipulação e análise de dados
PyArrow: Biblioteca para leitura e escrita de arquivos parquet
Spotipy: Biblioteca para acesso a API do Spotify

Use o comando abaixo para instalar os pacotes necessários:
```bash
$ pip install pandas pyarrow spotipy
```

Ou instale as dependências diretamente do arquivo `requirements.txt`:
```bash
$ pip install -r requirements.txt
```

### Configuração da API do Spotify com um .env

Para acessar a API do Spotify é necessário criar um aplicativo no [Spotify for Developers](https://developer.spotify.com/dashboard/applications) e obter as credenciais de acesso.

Crie um arquivo `.env` na raiz do projeto e adicione as seguintes variáveis de ambiente:
```bash
SPOTIPY_CLIENT_ID=seu_client_id
SPOTIPY_CLIENT_SECRET=sua_client_secret
```

In [20]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [42]:
env_vars = {}

# Le o arquivo .env
with open('.env') as f:
    env_vars = dict(
        tuple(line.replace('"', '').replace("'", '').strip().split('=', 1)) for line in f
    )

# Cria o objeto de autenticação
client_credentials_manager = SpotifyClientCredentials(
    client_id=env_vars['SPOTIPY_CLIENT_ID'],
    client_secret=env_vars['SPOTIPY_CLIENT_SECRET']
)

In [ ]:
# Configurando as credenciais do cliente
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Função para obter as 100 playlists mais populares de música independente do Brasil
def get_playlists(query = '', n = 1, offset = 50, market = 'BR'):
    all_tracks = 0
    all_playlists = []
    for i in range(n):
        try:
            playlists = sp.search(query, limit=offset, offset=i*offset, type='playlist', market = market)
            # Printando o nome das playlists
            for item in playlists['playlists']['items']:
                play_tracks = 0
                print(f"Playlist encontrada: {item['name']}")
                all_playlists.append(item)
                # Obtendo as tracks da playlist
                play_id = item['id']
                print(f"Obtendo músicas da playlist: {play_id}")
                tracks = get_playlist_tracks(play_id, market = market)
                # Salvando o objeto de tracks em um arquivo parquet
                df = pd.DataFrame(tracks)
                # Convertendo o DataFrame para um Arrow Table
                table = pa.Table.from_pandas(df)
                # Salvando o arquivo parquet
                pq.write_table(table, f'tracks_{play_id}.parquet')
                # Incrementando o contador de músicas
                play_tracks += len(tracks)
                print(f"Músicas encontradas: {play_tracks}")
                print("-------------------------------------------------------")
                all_tracks += play_tracks
        except Exception as e:
            print("Erro ao obter playlists: ", e)
            continue

    # Salvando o objeto de playlists em um arquivo parquet
    df = pd.DataFrame(all_playlists)
    # Convertendo o DataFrame para um Arrow Table
    table = pa.Table.from_pandas(df)
    # Salvando o arquivo parquet
    pq.write_table(table, f'playlists_{query}.parquet')
    print(f"Playlists encontradas: {len(all_playlists)}")
    print(f"Musicas encontradas: {all_tracks}")
    return all_playlists

# Função para obter todas as músicas de uma playlist
def get_playlist_tracks(playlist_id, limit = 100, market = 'BR'):
    all_tracks = []
    try:
        results = sp.playlist_tracks(playlist_id, limit = limit, market = market)
        while results['next']:
            results = sp.next(results)
            all_tracks.extend(results['items'])
    except Exception as e:
        print("Erro ao obter músicas da playlist: ", e)
    return all_tracks

# Obtendo as n*offset playlists retornadas pela função get_playlists
top_playlists = get_playlists(query = 'artistas independentes brasil 2023', n = 50, offset = 50, market = 'BR')


In [ ]:
## # Leitura do arquivo parquet
## table = pq.read_table('playlists_independente brasil.parquet')
## df = table.to_pandas()
## print(df)
## 
## # Exportando o DataFrame para um arquivo CSV
## df.to_csv('playlists_independente brasil.csv', index=False)